In [1]:
import pandas as pd
import numpy as np
import os
import sys
import plotly
import plotly.graph_objs as go
    
%load_ext autoreload
%autoreload 1

pd.set_option("display.max_columns",201)
pd.set_option("display.max_colwidth",101)
pd.set_option("display.max_rows",500)

In [2]:
from arctic import Arctic, CHUNK_STORE

conn = Arctic('10.213.120.5')
conn.initialize_library('entsoe', lib_type=CHUNK_STORE)
conn.list_libraries()
lib = conn['entsoe']

c:\users\a0743104\appdata\local\programs\python\python38-32\lib\site-packages\arctic\_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
c:\users\a0743104\appdata\local\programs\python\python38-32\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel
Library created, but couldn't enable sharding: no such command: 'enablesharding'. This is OK if you're not 'admin'


In [3]:
# Input country

country = input("Enter the perimeter (DE/FR/BE/ES/IT/PL) : ")

Enter the perimeter (DE/FR/BE/ES/IT/PL) : DE


In [4]:
# function to change timezone from UTC to local time

def changing_timezone(x):
    ts = x.index.tz_localize('utc').tz_convert('Europe/Brussels')
    y = x.set_index(ts)
    return y.tz_localize(None)

In [5]:
# define dates

from datetime import datetime
from datetime import timedelta
from datetime import date

ref_date = datetime(year=2019, month=1, day=1).date()
start_date = ref_date + timedelta(days = - 1)

end_date = date.today().replace(day=1)

In [8]:
# Read Spot price

read = 'DayAheadPrices'
prefix = read + '_' + country 

df_DA_price = lib.read(prefix, chunk_range=pd.date_range(start_date, end_date))

# changing timezones 
df_DA_price = changing_timezone(df_DA_price)

In [6]:
# Read installed capacity data

read = 'InstalledGenerationCapacityAggregated'
prefix = read + '_' + country 

df_inst_cap = lib.read(prefix, chunk_range=pd.date_range(start_date, end_date))

In [7]:
# converting annual indexes to hourly indexes

date_range = pd.date_range(start_date, end_date, freq='h')
df_inst_cap=df_inst_cap.reindex(date_range, method='ffill')

# changing timezones 
df_inst_cap = changing_timezone(df_inst_cap)

In [8]:
prefix = read + ' ' + country 
df_RES = pd.DataFrame()
df_RES = df_inst_cap[[prefix+' '+'Solar',prefix+' '+'Wind Onshore']]
try:
    df_RES[prefix+' '+'Wind Offshore'] = df_inst_cap[[prefix+' '+'Wind Offshore']]
except KeyError:
    pass

<ipython-input-8-9e720c52bd3a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RES[prefix+' '+'Wind Offshore'] = df_inst_cap[[prefix+' '+'Wind Offshore']]


In [9]:
# Read power generation data

read = 'AggregatedGenerationPerType'
prefix = read + '_' + country 

df_gen = lib.read(prefix,chunk_range=pd.date_range(start_date, end_date))

# convert 15 min data to hourly data
df_gen = df_gen.resample('H').mean()

# changing timezones 
df_gen = changing_timezone(df_gen)

In [13]:
read = ['InstalledGenerationCapacityAggregated','ActualGenerationOutput']
df = [df_inst_cap, df_gen]
var = ['Solar','Wind Onshore', 'Wind Offshore']

df_RES = pd.DataFrame()

k = 0
for i in read:
    prefix = i + ' ' + country 
    for j in var:
        try:
            df_RES = pd.concat([df_RES,df[k][prefix+' '+j]],axis=1)
        except KeyError:
            pass 
    k+=1
    
df_RES.index = pd.to_datetime(df_RES.index)

In [14]:
df_DA_price =df_DA_price[~df_DA_price.index.duplicated()]
df_RES =df_RES[~df_RES.index.duplicated()]

In [15]:
df_merge = pd.DataFrame()
df_merge = df_merge.append(df_DA_price)
for i in var:
    try:
        df_merge = pd.merge(df_merge,df_RES[prefix+' '+i],how='outer',right_index=True, left_index=True)
        df_merge['Tot_Cap_Price'+' '+i] = df_merge['DayAheadPrices_'+country]*df_RES[prefix+' '+i]
    except KeyError:
        pass

In [16]:
df_data= df_merge.iloc[(df_merge.index.year >=2016)&(df_merge.index.date<end_date)]

In [17]:
df_data_daily = df_data.groupby(df_data.index.date).mean()

In [18]:
for i in var:
    try:
        df_data_daily['Cap_Price' +'_'+i] = df_data_daily['Tot_Cap_Price' +' '+ i]/df_data_daily[prefix + ' ' + i]
    except KeyError:
        pass

In [19]:
df_data_monthly = df_data.groupby([(df_data.index.year),(df_data.index.month)]).mean()

In [20]:
for i in var:
    try:
        df_data_monthly['Cap_Price' +'_'+i] = df_data_monthly['Tot_Cap_Price' +' '+ i]/df_data_monthly[prefix + ' ' + i]
    except KeyError:
        pass

In [22]:
df_data_monthly['Qf_Solar'] = df_data_monthly['Cap_Price_Solar']/df_data_monthly['DayAheadPrices_'+country]

In [23]:
df_data_monthly['Qf_Wind Onshore'] = df_data_monthly['Cap_Price_Wind Onshore']/df_data_monthly['DayAheadPrices_'+country]

In [24]:
try:
    df_data_monthly['Qf_Wind Offshore'] = df_data_monthly['Cap_Price_Wind Offshore']/df_data_monthly['DayAheadPrices_'+country]
except KeyError:
    pass


In [43]:
try:
    df_final = df[['DayAheadPrices_'+country,'Cap_Price_Solar','Qf_Solar', 'Cap_Price_Wind Onshore','Qf_Wind Onshore','Cap_Price_Wind Offshore', 'Qf_Wind Offshore']]
except KeyError:
    df_final = df[['DayAheadPrices_'+country,'Cap_Price_Solar','Qf_Solar', 'Cap_Price_Wind Onshore','Qf_Wind Onshore']]
df_final = df_final.transpose()

In [40]:
import calendar 

df = df_data_monthly

#Use calendar library for abbreviations and order
dd=dict((enumerate(calendar.month_abbr)))

#rename level zero of multiindex
df = df.rename(index=dd,level=1)

#Create calendar month data type with order for sorting
cal_dtype = pd.CategoricalDtype(list(calendar.month_abbr), ordered=True)

#Change the dtype of the level zero index
df.index = df.index.set_levels(df.index.levels[1].astype(cal_dtype), level=1)


DayAheadPrices_ES  ActualGenerationOutput ES Solar  \
2016 Jan              36.53                           524.84   
     Feb              27.50                           829.09   
     Mar              27.83                          1403.32   
     Apr              24.11                          1495.70   
     May              25.77                          1670.93   
     Jun              38.90                          2309.87   
     Jul              40.53                          2240.46   
     Aug              41.16                          2158.95   
     Sep              43.59                          1783.26   
     Oct              52.84                          1080.16   
     Nov              56.13                           728.48   
     Dec              60.49                           632.69   
2017 Jan              71.49                           803.73   
     Feb              51.74                           744.95   
     Mar              43.21                          1315.81   
     Apr              43.69                          1808.28   
     May              47.11                          1921.31   
     Jun              50.22                          2170.54   
     Jul              48.63                          2227.13   
     Aug              47.46                          1951.10   
     Sep              49.15                          1881.08   
     Oct              56.79                          1371.27   
     Nov              59.19                           976.77   
     Dec              57.94                           726.95   
2018 Jan              49.98                           717.84   
     Feb              54.88                          1075.43   
     Mar              40.18                          1087.71   
     Apr              42.67                          1420.64   
     May              54.92                          1729.90   
     Jun              58.46                          1878.86   
     Jul              61.88                          2396.38   
     Aug              64.33                          2047.97   
     Sep              71.27                          1633.39   
     Oct              65.10                          1141.84   
     Nov              61.97                           608.82   
     Dec              61.81                           707.30   
2019 Jan              61.99                           887.15   
     Feb              54.01                          1321.74   
     Mar              48.82                          1702.66   
     Apr              50.41                          1417.60   
     May              48.39                          2208.48   
     Jun              47.19                          2366.43   
     Jul              51.46                          2367.63   
     Aug              44.96                          2487.88   
     Sep              42.11                          1849.50   
     Oct              47.17                          1494.08   
     Nov              42.19                           827.83   
     Dec              33.80                           796.53   
2020 Jan              41.10                           975.29   
     Feb              35.87                          1728.03   
     Mar              27.74                          1744.04   
     Apr              17.65                          1927.13   
     May              21.25                          3033.97   
     Jun              30.62                          3527.00   
     Jul              34.64                          3686.74   
     Aug              36.20                          3496.65   
     Sep              41.96                          2680.07   
     Oct              36.61                          2216.13   
     Nov              41.94                          1256.87   
     Dec              41.97                          1072.22   
2021 Jan              60.17                          1238.78   

          Tot_Cap_Price Solar  ActualGene

In [44]:
df_final

2016                                                  \
                          Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   
DayAheadPrices_ES       36.53 27.50 27.83 24.11 25.77 38.90 40.53 41.16 43.59   
Cap_Price_Solar         39.61 30.81 27.30 24.54 27.18 39.26 42.11 42.45 44.57   
Qf_Solar                 1.08  1.12  0.98  1.02  1.05  1.01  1.04  1.03  1.02   
Cap_Price_Wind Onshore  31.55 23.63 24.41 22.19 22.80 36.95 39.79 40.13 42.89   
Qf_Wind Onshore          0.86  0.86  0.88  0.92  0.88  0.95  0.98  0.97  0.98   
Cap_Price_Wind Offshore   nan   nan   nan   nan   nan   nan   nan   nan   nan   
Qf_Wind Offshore          nan   nan   nan   nan   nan   nan   nan   nan   nan   

                                           2017                                \
                          Oct   Nov   Dec   Jan   Feb   Mar   Apr   May   Jun   
DayAheadPrices_ES       52.84 56.13 60.49 71.49 51.74 43.21 43.69 47.11 50.22   
Cap_Price_Solar         52.85 59.19 62.52 74.82 54.36 44.39 43.75 47.78 51.69   
Qf_Solar                 1.00  1.05  1.03  1.05  1.05  1.03  1.00  1.01  1.03   
Cap_Price_Wind Onshore  50.95 53.20 58.96 71.18 49.21 40.59 40.91 45.13 48.55   
Qf_Wind Onshore          0.96  0.95  0.97  1.00  0.95  0.94  0.94  0.96  0.97   
Cap_Price_Wind Offshore   nan   nan   nan   nan   nan   nan   nan   nan   nan   
Qf_Wind Offshore          nan   nan   nan   nan   nan   nan   nan   nan   nan   

                                                             2018              \
                          Jul   Aug   Sep   Oct   Nov   Dec   Jan   Feb   Mar   
DayAheadPrices_ES       48.63 47.46 49.15 56.79 59.19 57.94 49.98 54.88 40.18   
Cap_Price_Solar         49.92 48.86 49.48 57.92 60.48 64.60 53.06 55.91 40.54   
Qf_Solar                 1.03  1.03  1.01  1.02  1.02  1.11  1.06  1.02  1.01   
Cap_Price_Wind Onshore  48.17 46.24 47.20 54.89 56.83 54.37 46.80 53.61 38.65   
Qf_Wind Onshore          0.99  0.97  0.96  0.97  0.96  0.94  0.94  0.98  0.96   
Cap_Price_Wind Offshore   nan   nan   nan   nan   nan   nan   nan   nan   nan   
Qf_Wind Offshore          nan   nan   nan   nan   nan   nan   nan   nan   nan   

                                                                               \
                          Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec   
DayAheadPrices_ES       42.67 54.92 58.46 61.88 64.33 71.27 65.10 61.97 61.81   
Cap_Price_Solar         42.90 54.31 58.90 63.21 66.38 72.70 67.41 64.33 64.30   
Qf_Solar                 1.01  0.99  1.01  1.02  1.03  1.02  1.04  1.04  1.04   
Cap_Price_Wind Onshore  39.79 50.63 57.14 61.12 63.38 70.05 62.63 60.47 60.60   
Qf_Wind Onshore          0.93  0.92  0.98  0.99  0.99  0.98  0.96  0.98  0.98   
Cap_Price_Wind Offshore   nan   nan   nan   nan   nan   nan   nan   nan   nan   
Qf_Wind Offshore          nan   nan   nan   nan   nan   nan   nan   nan   nan   

                         2019                                                  \
                          Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   
DayAheadPrices_ES       61.99 54.01 48.82 50.41 48.39 47.19 51.46 44.96 42.11   
Cap_Price_Solar         64.36 54.63 49.28 52.44 48.68 47.01 52.88 45.71 43.00   
Qf_Solar                 1.04  1.01  1.01  1.04  1.01  1.00  1.03  1.02  1.02   
Cap_Price_Wind Onshore  60.47 51.29 46.69 47.41 45.83 46.01 50.74 43.83 39.19   
Qf_Wind Onshore          0.98  0.95  0.96  0.94  0.95  0.97  0.99  0.97  0.93   
Cap_Price_Wind Offshore   nan   nan   nan   nan   nan   nan   nan   nan   nan   
Qf_Wind Offshore          nan   nan   nan   nan   nan   nan   nan   nan   nan   

                                           2020                                \
                          Oct   Nov   Dec   Jan   Feb   Mar   Apr   May   Jun   
DayAheadPrices_ES       47.17 42.19 33.80 41.10 35.87 27.74 17.65 21.25 30.62   
Cap_Price_Solar         46.86 42.58 36.01 41.66 35.15 27.91 16.86 20.88 30.20   
Qf_Solar                 0.99  1.01  1.07  1.01  0.

In [28]:
df_final.to_html('RES_Capture_Prices_'+country+'.html')

In [45]:
filename = 'Hist_cap_price.xlsx'

In [49]:
import openpyxl

df_final.to_excel(filename, sheet_name=country)